# Index

## 1. Introduce Dataset
----------------------------
##### 1-1. cifar-100 
##### 1-2. Tiny-Imagenet

## 2. Introduce Algorithm
----------------------------
##### 2-1. Cutout
##### 2-2. Mixup
#### 2-3. *CutMix*
##### 2-4. Compare

## 3. Introduce Models
----------------------------
##### 3-1. ResNet-50
##### 3-2. PyramidNet-110 & 200

## 4. Introduce Training Process
----------------------------
##### 4-1. Hyper-Params
##### 4-2. Loss functions per every methods
##### 4-3. Savepoint

## 5. Show results
----------------------------
##### 5-1. introduce every experiments
##### 5-2. Compare Results with paper
